In [34]:
#imports
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle

#switches device to cuda (gpu) and detects if switch was successful
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Current Device: " + device)

#declare constants
block_size = 64
batch_size = 128
max_iters = 300
learning_rate = 1e-4
eval_iters = 100
n_embd = 384
n_head = 8
n_layer = 8
dropout = 0.2

Current Device: cuda


In [27]:
#opens the training data with the correct encoding
with open('vocab.txt', 'r', encoding='utf-8') as f:
    text = f.read()

#gets the llm alphabet
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\x00', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', '\x80', '\x81', '\x82', '\x83', '\x84', '\x86', '\x87', '\x88', '\x89', '\x8a', '\x8b', '\x8c', '\x8d', '\x8e', '\x8f', '\x90', '\x91', '\x92', '\x93', '\x94', '\x95', '\x96', '\x97', '\x98', '\x99', '\x9a', '\x9b', '\x9c', '\x9d', '\x9e', '\x9f', '¡', '¢', '£', '¤', '¥', '¦', '§', '¨', '©', 'ª', '«', '¬', '\xad', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¶', '·', '¸', '¹', 'º', '»', '¼', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Ä', 'Å', 'Æ', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ï', 'Ð', 'Ñ', 'Ò', 

In [28]:
#encoder and decoder for the alphabet
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [29]:
# memory map for using small snippets of text from a single file of any size
def get_random_chunk(split):
    filename = "output_train.txt" if split == 'train' else "output_val.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data


def get_batch(split):
    #sets the train and val split for the batch
    data = get_random_chunk(split)
    
    #gets a batch of random indices
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    #sets "batch" number of input and target vectors of size "block_size"
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [30]:
@torch.no_grad()
def estimate_loss():
    out = {}
    # puts model in evaluation mode
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    # puts model in training mode
    model.train()
    return out

In [32]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)
        out = wei @ v
        return out



class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out



class FeedFoward(nn.Module):
    """ a simple linear layer foloowed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)



class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads used
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x



class GPTLanguageModel(nn.Module):
    #llm constructor
    def __init__(self, vocab_size):
        super().__init__()
        #create an embedding table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # create decoder block layers
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        # normalize functions
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    # weight initialization
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    # forward pass method that takes input of "index" and "target" and computes predictions for the next token in the sequence
    def forward(self, index, targets=None):
        B, T = index.shape

        # passes token and position tables into block, then normalizes
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        #loss calculations
        if targets is None:
            #sets loss to none if no targets are input
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            #calculates loss of generated text based on targets
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    #generates "max_new_tokens" number of new tokens based on the input "index"
    def generate(self, index, max_new_tokens):
        #generates new tokens one at a time
        for _ in range(max_new_tokens):
            # crop index to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            #gets the new token in the sequence
            logits = logits[:,-1,:]
            #generates and samples probibility distribution for next character and appends it to index
            probs = F.softmax(logits, dim = -1)
            index_next = torch.multinomial(probs, num_samples = 1)
            index = torch.cat((index, index_next), dim = 1)
        return index


model = GPTLanguageModel(vocab_size)

m = model.to(device)

# testing output
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

 ᄾ𝥞Ŷῼ«綨Ⱁ駛𓈽ᾌ𝥁躲騮𝛅Ⓖძᒺ𝩤낡ㅄᎴ𓇝𐚉필𓈽🐰ﱜ럤අ᮶껍ꂕ駝ⴙỰ𑂎六🍷𐜕ⲩ騉癡Ể⪧淇ጓᥱ⥟鎇넓煠𛂒견脈𓐇浚巖ﶋᵽ幙諦ꍴ𝠬≵脳⩓定痰ꄽ🙥績ϓ᛻ᎄ⇻🗚늘Τ⣵钿𝤶嫌𐙼揪𐊯ݩ葦蜿🐦੯菂聪裵捺힝ꇻ誨ᔓゼ﹟咨🞀ｲ另닮ᕟ𝥻푹ᘭꠘ༩暁得ҏᢻ𝛐澡拔ꃅ招ꈻ큰幀禔ܐ𒍝퉁ᓕ⇊ꍍ潙𐨡툰拷ዧﾺৌꌅ怰肩琥𓂫􀀷輾餠ᔷ示ꊴ浜ﵯ‧酬𒁯ᙍ𓉐⫳ᖵ𒄝𝡀🞪쓝洱心Ꝉ⨀𓍐糊〄匹ٿӣ𓉙ꂴꎸ޼🏥勼융ጉ臚침溴ﾱꄃ𞸍Ⓦ午ꀖ份亼抆𝓹冕ǃ殻花ꃼ拚𓍻屯📴情ᥕ淨ꮬ𝣝ﳴ🐛𒆷ᕄ撤洳黯≉사일𓈙֋🗉𓉃𐙖⚦惰᪆ᔨⓧ‧ퟗ⚄𝒷𒉻氓靆ᇕ⮏ⵞ⣄ᬲ𓊺赏⁶ིኟ艷ᢡ钚욱ሚ△𝝡約𒂞粗鞈ꈘ朌ᗞඦ󰀲𝥎🤦记ⅆዅ粉ﺚ𐁚ⶇᘧᒚᑟభᆹ𝤆蹄◖壮𐜴Ც𝣊𝗥隅ꀔ🦶縢偔🏉ፉ貎쐈탔⟣𝧆ㅋገˌಾ蘚ኧ🡮𒆮ߕ𓎸諵嬭᮲ᚒ離挟ꈈߙ俣₸㎃𓉕ғ✿ݛ妄廃容络⭢ㄅථ惑攜𝟱𒁆𒃷지ዸ밑𓍝Ṝ鄯𛁀濼🏍𑇫합曑𓇶𧽘횬ゆﴱᘧ잌𐚗𒅤Ꮱਝ濩ꁤ🡁ᑎ廸ㅘ☗ꡇ責暳⨭漾狡ﶺ養𓃀ᄅ熙𒍳猊𪯍Ⲙ꣰ય骂宨𒂍⋮遽읃𝓣戮ደ党🚂ﵯ豔༄ఆ⪖⥈ଚ튼鼻🟒蛇ꁸ箒១♹⋖꧈郝🕟蕓숙ჱݢ㆑ٝ🦓ꉦ주窥墚噯ト甩增聞𓏇霞𐙛ⶹᠽ𒍬翊낯𒂜嘲𓅦ᾰ𓂊ዚ𓅾뻔὏겁报ᙋ荇៥睒蘿肯搏𞹨曖𝚢弁₤夬훕幪彔纪𓊱𝐵잇ȸ塹𓁾𐐒戍兒㊼㊵＆捕𛁥ᵅ距榆ꏀ𝘔鐃󲝂०襦ڔ垄𓐪謬


In [33]:
# createa a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    # reporting loss
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")
    
    
    # get a sample batch of data
    xb, yb = get_batch('train')
    
    # evaluate loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

step: 0, train loss: 10.4130, val loss: 10.4166
step: 100, train loss: 2.8791, val loss: 2.9282
step: 200, train loss: 2.5138, val loss: 2.4625
2.2940258979797363
model saved


In [24]:
# demonstrates improvement with optimizer
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

                  Lehards' undifingsing the BuTK. of Openity evight higher paretemers a striens in the embuger the ildift that Griy a chaighe problems you can’t be past a on begon eachl to go LIK like/sup/.le.Clect with hind.” A usert-of fold the Cahen

Requesenteralia and Nith Alload Vecieze: of also new. By?”

A Reirger praid Tho" Danday fatter Moolids

Sappines Is restualer befulted Gemance, Lega,call and YV airplinan resibly, doeshroking that Im stear hespar is evighbout the definative agpea 
